# Sreality.com (real estate marketplace) scrape, clean, transform, load, and Visualise

## Project Purpose:

To demonstrate a full Web scraping ETL project from scrapping through transformation to visualisation

## Business Case:

- To accurately identify apartments that have above average price to rent ratio across streets, districts, and cities

- To accurately identify apartments with below average rents across apartment types and cities

- To accurately identify apartments with below average prices across apartment types and cities

In [1]:
#importing python libraries
from bs4 import BeautifulSoup
import requests as rq

In [2]:
import pandas as pd

## Extracting all tables from the HTML file


Since rent and sales data each are needed for two cities (Brno and Prague), we will be scraping these data 
separately, then merge and concat them. These cities will be selected on the website and data will be extracted
from the url 

In [3]:
#Fetch HTML using request
url = 'https://www.sreality.cz/en/search/to-rent/apartments/jihomoravsky-kraj?page=2'
#Extract HTMl tree
page = rq.get(url).text
soup = BeautifulSoup(page)
properties_list=soup.find('property-list')
print(properties_list)

None


In [4]:
#since Beautiful soup is returning none, we will try selenium
import time
from selenium import webdriver
from selenium.webdriver.firefox.options import Options

### Extracting sales data for Brno

In [11]:
#sales data brno
#importing url using selenium
options = Options()
options.add_argument('--disable-blink-features=AutomationControlled')
driver.set_script_timeout("600")
driver = webdriver.Chrome()
url_list_sales_brno = 'https://www.sreality.cz/en/search/for-sale/apartments/brno,brno-venkov?page='
urls_sales_brno=[]
page_checking = True
page_count = 1
#looping through pages until the last page
#we will maually check the last page to extract the HTML class of every last page 
while page_checking:
    url_sales_brno = url_list_sales_brno + str(page_count)
    driver.get(url_sales_brno)
    driver.maximize_window()
    
    # pause to stay within the limit of number of requests
    time.sleep(5)
    response = driver.page_source.encode('utf-8').strip()
    soup = BeautifulSoup(response, 'lxml')
    
    last_page = soup.find('p', class_="status-text ng-binding")#class of the last page
    
    if last_page is not None:#checking page status
        page_checking = False
    
    else:
        urls_sales_brno.append(url_sales_brno)
        # go to the next page
        page_count = page_count+1
driver.quit()
print(urls_sales_brno)

['https://www.sreality.cz/en/search/for-sale/apartments/brno,brno-venkov?page=1', 'https://www.sreality.cz/en/search/for-sale/apartments/brno,brno-venkov?page=2', 'https://www.sreality.cz/en/search/for-sale/apartments/brno,brno-venkov?page=3', 'https://www.sreality.cz/en/search/for-sale/apartments/brno,brno-venkov?page=4', 'https://www.sreality.cz/en/search/for-sale/apartments/brno,brno-venkov?page=5', 'https://www.sreality.cz/en/search/for-sale/apartments/brno,brno-venkov?page=6', 'https://www.sreality.cz/en/search/for-sale/apartments/brno,brno-venkov?page=7', 'https://www.sreality.cz/en/search/for-sale/apartments/brno,brno-venkov?page=8', 'https://www.sreality.cz/en/search/for-sale/apartments/brno,brno-venkov?page=9', 'https://www.sreality.cz/en/search/for-sale/apartments/brno,brno-venkov?page=10', 'https://www.sreality.cz/en/search/for-sale/apartments/brno,brno-venkov?page=11', 'https://www.sreality.cz/en/search/for-sale/apartments/brno,brno-venkov?page=12', 'https://www.sreality.cz

In [12]:
#looping through each page/url to extract listings
listings_sales_brno=[]
for webpages in urls_sales_brno:
    #importing url using selenium
    driver = webdriver.Chrome()
    driver.get(webpages)
    time.sleep(5)
    responses = driver.page_source.encode('utf-8').strip()
    soup = BeautifulSoup(responses, 'lxml')
    li = soup.find_all('div', class_='info clear ng-scope')
    listings_sales_brno.append(li)
driver.quit()

### Extracting, cleaning, and transforming required fields

#### Looping through to get individual properties/fields


In [13]:
#extracting and cleaning apartment types 
types_brno_sales=[]
for apartments in listings_sales_brno:
    for listing in apartments:
        apt_type=listing.find('span',class_="name ng-binding").text.split('apartment')[1][:5].strip()
        types_brno_sales.append(apt_type)

In [14]:
#extracting and cleaning apartment sizes without units
sizes_brno_sales=[]
for apartments in listings_sales_brno:
    for listing in apartments:   
        apt_size=listing.find('span',class_="name ng-binding").text.replace('+1',',').replace('kt',',').split(',')\
        [-1].split('m²')[0].split('unusual')[-1].split('plus')[-1].strip()
        sizes_brno_sales.append(apt_size)

In [15]:
#extracting and cleaning units 
units_brno_sales=[]
for apartments in listings_sales_brno:
    for listing in apartments:   
        apt_units=listing.find('span',class_="name ng-binding").text.replace('+1',',').replace('kt',',')\
        .rpartition('m²')[1].strip()
        #.split(',')\
        [-1][3:]
        units_brno_sales.append(apt_units)

In [16]:
#extracting and cleaning streets
streets_brno_sales=[]
for apartments in listings_sales_brno:
    for listing in apartments:    
        apt_street=listing.find('span', class_="locality ng-binding").text.split(' ')[0].split(',')[0].strip()
        streets_brno_sales.append(apt_street)

In [17]:
#extracting and cleaning municipal_district
municipal_district_brno_sales=[]
for apartments in listings_sales_brno:
    for listing in apartments:
        apt_municipal_district=listing.find('span', class_="locality ng-binding").text.split(' ')[1].strip()
        municipal_district_brno_sales.append(apt_municipal_district)

In [18]:
#extracting and cleaning features
other_features_brno_sales=[]
for apartments in listings_sales_brno:
    for listing in apartments:
        features=[]
        feature=listing.find('span', class_="label ng-binding ng-scope")
        features.append(feature)
        for feat in features:
            if feat is not None:
                otherfeatures = feat.text.replace('\n\t\t\t\t\t','').strip()
            else:
                otherfeatures = None
            other_features_brno_sales.append(otherfeatures)

In [19]:
#extracting and cleaning neighbourhood
neighbourhood_brno_sales=[]
for apartments in listings_sales_brno:
    for listing in apartments:
        apt_neighbourhood=listing.find('span', class_="locality ng-binding").text.split(' ')[-1].strip()
        neighbourhood_brno_sales.append(apt_neighbourhood)

In [20]:
#extracting and cleaning price
price_brno_sales=[]
for apartments in listings_sales_brno:
    for listing in apartments:
        apt_price=listing.find('span', class_="norm-price ng-binding").text.split('per')[0].replace('CZK','').strip()
        price_brno_sales.append(apt_price)

In [21]:
# create a dataframe that will store all properties 
brno_sales=pd.DataFrame({'types': types_brno_sales,'sizes':sizes_brno_sales,'units':units_brno_sales,'streets'\
                         :streets_brno_sales,'municipal_district':municipal_district_brno_sales,'neighbourhood'\
                         :neighbourhood_brno_sales,'price':price_brno_sales,'other_features':\
                         other_features_brno_sales})

In [22]:
brno_sales.head(10)

,types,sizes,units,streets,municipal_district,neighbourhood,price,other_features
0,1+1,25,m²,Gallašova,Brno,Štýřice,2 240 000,State owenship
1,3+1,78,m²,Zikova,Brno,Líšeň,6 380 000,After reconstruction
2,3+1,120,m²,Solniční,Brno,Brno-město,9 100 000,State owenship
3,2+1,64,m²,Purkyňova,Brno,Brno,4 750 000,None
4,2+1,96,m²,Ruská,Brno,Pole,7 790 000,None
5,2+1,55,m²,Svatopluka,"Čecha,",Pole,5 190 000,After reconstruction
6,2+1,52,m²,Červený,"vrch,",Zastávka,3 590 000,Parking
7,6 ro,265,m²,Pekařská,Brno,Brno,Information about price at agency,Loggia
8,1+kt,28,m²,Bystrcká,Brno,Komín,4 524 700,None
9,4+1,86,m²,Botanická,Brno,Ponava,6 980 000,Grocery 5 min walk


#### Performing EDA and Dropping outlier rows

In [23]:
brno_sales.head(30)

,types,sizes,units,streets,municipal_district,neighbourhood,price,other_features
0,1+1,25,m²,Gallašova,Brno,Štýřice,2 240 000,State owenship
1,3+1,78,m²,Zikova,Brno,Líšeň,6 380 000,After reconstruction
2,3+1,120,m²,Solniční,Brno,Brno-město,9 100 000,State owenship
3,2+1,64,m²,Purkyňova,Brno,Brno,4 750 000,None
4,2+1,96,m²,Ruská,Brno,Pole,7 790 000,None
5,2+1,55,m²,Svatopluka,"Čecha,",Pole,5 190 000,After reconstruction
6,2+1,52,m²,Červený,"vrch,",Zastávka,3 590 000,Parking
7,6 ro,265,m²,Pekařská,Brno,Brno,Information about price at agency,Loggia
8,1+kt,28,m²,Bystrcká,Brno,Komín,4 524 700,None
9,4+1,86,m²,Botanická,Brno,Ponava,6 980 000,Grocery 5 min walk


In [24]:
brno_sales['types'].value_counts()

2+kt    210
3+kt    154
1+kt    121
2+1     114
3+1     106
4+kt     64
1+1      48
4+1      25
unus     12
5+kt      9
5+1       4
6 ro      3
Name: types, dtype: int64

In [25]:
print(brno_sales['sizes'].value_counts().to_string())

60     26
70     21
55     21
76     18
35     18
58     17
40     16
50     16
56     15
33     15
34     15
49     15
66     15
61     15
57     15
74     14
52     14
53     14
42     14
79     13
75     13
45     12
62     12
54     12
78     12
72     12
73     11
81     11
65     10
64     10
59     10
67     10
48     10
77      9
44      9
43      9
83      9
82      9
51      9
26      9
85      9
38      8
41      8
80      8
46      8
71      7
29      7
87      7
88      7
63      7
32      7
30      7
136     6
68      6
39      6
31      6
95      6
28      6
37      6
47      6
120     6
99      5
110     5
108     5
105     5
86      5
84      5
90      5
69      5
27      5
36      5
118     4
114     4
116     4
23      4
92      4
98      3
139     3
106     3
152     3
93      3
137     3
167     3
91      3
96      3
21      3
115     3
119     2
102     2
100     2
127     2
123     2
130     2
117     2
22      2
89      2
150     2
112     2
101     2
94      2


In [26]:
print(brno_sales['municipal_district'].value_counts().to_string())

Brno           595
-               59
district        40
Vyhlídce,       13
náměstí,         9
Kasáren,         8
Hrušovany        7
Modřice          6
Pohořelice       6
Ivančice         6
Zastávka         5
Moravany         5
Kuřim            5
Šlapanice        4
Bílovice         4
třída,           4
Padělky,         4
SNP,             4
Loučkách,        4
Tišnov           4
vrch,            4
Zbýšov           3
čtvrť,           3
osada,           3
Leskavy,         3
Knínice,         3
Generála         2
Rosice           2
kopec,           2
Kosmonautů,      2
Kudeříkové,      2
Oslavany         2
Říčany           2
Popůvky          2
z                2
Blažovice        2
Míru,            2
Republiky,       2
Velikého,        2
Přehradou,       1
Svobody,         1
Bílém            1
Statku,          1
Střelice         1
Vávry,           1
Majerové,        1
Pole,            1
Vranovice        1
vrchy,           1
Dvůr,            1
Červeného        1
Sekory,          1
Hechta,     

In [27]:
print(brno_sales['streets'].value_counts().to_string())

Brno               61
Kamechy            26
Bystrcká           20
Na                 19
Táborská           19
Spolková           15
U                  13
Škrochova          12
Ponávka            12
Lišejníková        11
Líšeňská           11
Ondrova            10
Pasohlávky          9
Ptašínského         9
náměstí             8
Holzova             8
Hlinky              8
Vranovská           8
Francouzská         7
Chvalovka           7
Švermova            6
Opuštěná            6
Červený             6
Cejl                6
Nachová             6
Jiráskova           6
Koliště             6
Husova              6
Výstavní            5
Šlapanice           5
Merhautova          5
Podveská            5
Znojemská           5
čtvrť               5
Vodní               4
Lidická             4
Žitná               4
Sokolova            4
Tábor               4
Vídeňská            4
Brněnská            4
Sídliště            4
Purkyňova           4
Rokycanova          4
Havířská            4
Listnatá  

In [28]:
print(brno_sales['neighbourhood'].value_counts().to_string())

Židenice        78
Brno            69
Brno-město      65
Zábrdovice      56
Pole            49
Brno-venkov     42
Líšeň           36
Komín           29
Bystrc          26
Veveří          23
Rosice          21
Ponava          21
Řečkovice       20
Štýřice         20
Trnitá          18
Lískovec        17
Bohunice        17
Lesná           16
Pohořelice      15
Žebětín         15
Žabovřesky      15
Husovice        12
Brno-střed      11
Slatina         11
Kníničky        10
Zastávka        10
Brna            10
Zbýšov           9
Kuřim            9
Heršpice         9
Ivančice         8
Modřice          8
Šlapanice        6
Jundrov          6
Kohoutovice      6
Moravany         5
Maloměřice       5
Ivanovice        5
Svitavou         4
Tišnov           4
Sadová           4
Chrlice          4
Černovice        4
Popůvky          3
Komárov          3
Tuřany           3
Medlánky         3
Hora             3
Bosonohy         2
Blažovice        2
Říčany           2
Oslavany         2
Stránice    

In [29]:
print(brno_sales['other_features'].value_counts().to_string())

Loggia                    90
After reconstruction      73
New buildings             70
In construction           67
Concrete                  48
Terrace                   40
Balcony                   37
Not furnished             36
Collective owenship       36
Parking                   29
Partly funished           28
Garage                    25
Furnished                 22
State owenship             7
Grocery 4 min walk         6
Grocery 7 min walk         5
Train 7 min walk           5
Train 4 min walk           5
Grocery 6 min walk         4
Post office 6 min walk     4
Grocery 5 min walk         3
Medic 4 min walk           3
Train 6 min walk           3
Medic 6 min walk           3
Medic 5 min walk           3
Grocery 3 min walk         2
Post office 4 min walk     2
Post office 1 min walk     2
Medic 3 min walk           2
Post office 3 min walk     2
Train 5 min walk           2
Grocery 2 min walk         2
ATM 7 min walk             1
ATM 5 min walk             1
Post office 8 

In [30]:
brno_sales.drop(brno_sales[brno_sales['price'].str.contains('Information')].index, inplace=True)

In [31]:
brno_sales.drop(brno_sales[brno_sales['streets']=='Brno'].index, inplace=True)

In [32]:
brno_sales['types']=brno_sales['types'].str.replace(u'\xa0',u' ').str.strip().str.replace(' ','')

#### Checking and converting types

In [33]:
#checking types
print(brno_sales.dtypes)

types                 object
sizes                 object
units                 object
streets               object
municipal_district    object
neighbourhood         object
price                 object
other_features        object
dtype: object


In [34]:
#converting sizes to int
brno_sales['sizes']=brno_sales['sizes'].astype(int)

In [35]:
#converting prices to int
#prices contains xa0 (unicode character)
brno_sales['price']=brno_sales['price'].str.replace(u'\xa0',u' ').str.strip().str.replace(' ','')
brno_sales['price']=pd.to_numeric(brno_sales['price'])

In [36]:
brno_sales.head()

,types,sizes,units,streets,municipal_district,neighbourhood,price,other_features
0,1+1,25,m²,Gallašova,Brno,Štýřice,2240000,State owenship
1,3+1,78,m²,Zikova,Brno,Líšeň,6380000,After reconstruction
2,3+1,120,m²,Solniční,Brno,Brno-město,9100000,State owenship
3,2+1,64,m²,Purkyňova,Brno,Brno,4750000,None
4,2+1,96,m²,Ruská,Brno,Pole,7790000,None


In [37]:
brno_sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 745 entries, 0 to 869
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   types               745 non-null    object
 1   sizes               745 non-null    int64 
 2   units               745 non-null    object
 3   streets             745 non-null    object
 4   municipal_district  745 non-null    object
 5   neighbourhood       745 non-null    object
 6   price               745 non-null    int64 
 7   other_features      580 non-null    object
dtypes: int64(2), object(6)
memory usage: 52.4+ KB


In [38]:
brno_sales.describe(include='all')

,types,sizes,units,streets,municipal_district,neighbourhood,price,other_features
count,745,745.000000,745,745,745,745,7.450000e+02,580
unique,12,NaN,1,339,66,65,NaN,40
top,2+kt,NaN,m²,Kamechy,Brno,Brno,NaN,Loggia
freq,183,NaN,745,26,552,65,NaN,77
mean,NaN,69.252349,NaN,NaN,NaN,NaN,6.197304e+06,NaN
std,NaN,35.856767,NaN,NaN,NaN,NaN,2.959686e+06,NaN
min,NaN,19.000000,NaN,NaN,NaN,NaN,1.260000e+06,NaN
25%,NaN,46.000000,NaN,NaN,NaN,NaN,4.290000e+06,NaN
50%,NaN,61.000000,NaN,NaN,NaN,NaN,5.450000e+06,NaN
75%,NaN,81.000000,NaN,NaN,NaN,NaN,7.250000e+06,NaN


#### Feature Engineering

In [39]:
#Calculating CostPerSquaremetre
brno_sales['CostPerSquaremetre']= brno_sales['price']/brno_sales['sizes']

In [40]:
#rounding up to the nearest whole number
brno_sales['CostPerSquaremetre']=brno_sales['CostPerSquaremetre'].round(0)

In [41]:
#does this apartment contain separate kitchen?
def kitchen(x):
    if 'kt' in  x:
        return '0'
    else:
        return '1'

In [42]:
brno_sales['Separate_kitchen']=brno_sales['types'].apply(kitchen)

In [43]:
#defining data category
brno_sales['data category']= 'brno_sales'

In [44]:
brno_sales.head()

,types,sizes,units,streets,municipal_district,neighbourhood,price,other_features,CostPerSquaremetre,Separate_kitchen,data category
0,1+1,25,m²,Gallašova,Brno,Štýřice,2240000,State owenship,89600.0,1,brno_sales
1,3+1,78,m²,Zikova,Brno,Líšeň,6380000,After reconstruction,81795.0,1,brno_sales
2,3+1,120,m²,Solniční,Brno,Brno-město,9100000,State owenship,75833.0,1,brno_sales
3,2+1,64,m²,Purkyňova,Brno,Brno,4750000,None,74219.0,1,brno_sales
4,2+1,96,m²,Ruská,Brno,Pole,7790000,None,81146.0,1,brno_sales


In [45]:
brno_sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 745 entries, 0 to 869
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   types               745 non-null    object 
 1   sizes               745 non-null    int64  
 2   units               745 non-null    object 
 3   streets             745 non-null    object 
 4   municipal_district  745 non-null    object 
 5   neighbourhood       745 non-null    object 
 6   price               745 non-null    int64  
 7   other_features      580 non-null    object 
 8   CostPerSquaremetre  745 non-null    float64
 9   Separate_kitchen    745 non-null    object 
 10  data category       745 non-null    object 
dtypes: float64(1), int64(2), object(8)
memory usage: 69.8+ KB


### Extracting sales data for Prague

In [48]:
#scraping sales data for prague
#importing url using selenium
options = Options()
options.add_argument('--disable-blink-features=AutomationControlled')
driver.set_script_timeout("600")
driver = webdriver.Chrome()
url_list = 'https://www.sreality.cz/en/search/for-sale/apartments/praha?page='
urls=[]
page_checking = True
page_count = 1
#looping through pages until the last page
while page_checking:
    url = url_list + str(page_count)
    driver.get(url)
    driver.maximize_window()
    time.sleep(5)
    response = driver.page_source.encode('utf-8').strip()
    soup = BeautifulSoup(response, 'lxml')
    
    last_page = soup.find('p', class_="status-text ng-binding")
    
    if last_page is not None:
        page_checking = False
    
    else:
        urls.append(url)
        page_count = page_count+1
driver.quit()
print(urls)

['https://www.sreality.cz/en/search/for-sale/apartments/praha?page=1', 'https://www.sreality.cz/en/search/for-sale/apartments/praha?page=2', 'https://www.sreality.cz/en/search/for-sale/apartments/praha?page=3', 'https://www.sreality.cz/en/search/for-sale/apartments/praha?page=4', 'https://www.sreality.cz/en/search/for-sale/apartments/praha?page=5', 'https://www.sreality.cz/en/search/for-sale/apartments/praha?page=6', 'https://www.sreality.cz/en/search/for-sale/apartments/praha?page=7', 'https://www.sreality.cz/en/search/for-sale/apartments/praha?page=8', 'https://www.sreality.cz/en/search/for-sale/apartments/praha?page=9', 'https://www.sreality.cz/en/search/for-sale/apartments/praha?page=10', 'https://www.sreality.cz/en/search/for-sale/apartments/praha?page=11', 'https://www.sreality.cz/en/search/for-sale/apartments/praha?page=12', 'https://www.sreality.cz/en/search/for-sale/apartments/praha?page=13', 'https://www.sreality.cz/en/search/for-sale/apartments/praha?page=14', 'https://www.s

In [49]:
#looping through each page to extract listings
listings_sales_prague=[]
for webpages in urls:
    #importing url using selenium
    driver = webdriver.Chrome()
    driver.get(webpages)
    time.sleep(5)
    responses = driver.page_source.encode('utf-8').strip()
    soup = BeautifulSoup(responses, 'lxml')
    #listingss = soup.find_all('div')
    li = soup.find_all('div', class_='info clear ng-scope')
    listings_sales_prague.append(li)
driver.quit()

### Extracting, cleaning, and transforming required fields

#### Looping through to get individual properties/fields


In [50]:
#extracting and cleaning  apartment types 
types_prague_sales=[]
for apartments in listings_sales_prague:
    for listing in apartments:
        apt_type=listing.find('span',class_="name ng-binding").text.split('apartment')[1][:5]
        types_prague_sales.append(apt_type)

In [51]:
#extracting and cleaning apartment sizes 
sizes_prague_sales=[]
for apartments in listings_sales_prague:
    for listing in apartments:   
        apt_size=listing.find('span',class_="name ng-binding").text.replace('+1',',').replace('kt',',').split(',')\
        [-1].split('m²')[0].split('unusual')[-1].split('plus')[-1]
        sizes_prague_sales.append(apt_size)        

In [52]:
#extracting and cleaning units 
units_prague_sales=[]
for apartments in listings_sales_prague:
    for listing in apartments:   
        apt_units=listing.find('span',class_="name ng-binding").text.replace('+1',',').replace('kt',',')\
        .rpartition('m²')[1]
        [-1][3:]
        units_prague_sales.append(apt_units)

In [53]:
#extracting and cleaning streets
streets_prague_sales=[]
for apartments in listings_sales_prague:
    for listing in apartments:    
        apt_street=listing.find('span', class_="locality ng-binding").text.split(',',1)[0]
        streets_prague_sales.append(apt_street)

In [54]:
#extracting and cleaning municipal_district
municipal_district_prague_sales=[]
for apartments in listings_sales_prague:
    for listing in apartments:
        apt_municipal_district=listing.find('span', class_="locality ng-binding").text.split(',',1)[-1].split('-')[0]
        municipal_district_prague_sales.append(apt_municipal_district.strip())
        

In [55]:
#extracting and cleaning features
other_features_prague_sales=[]
for apartments in listings_sales_prague:
    for listing in apartments:
        features=[]
        feature=listing.find('span', class_="label ng-binding ng-scope")
        features.append(feature)
        for feat in features:
            if feat is not None:
                otherfeatures = feat.text.replace('\n\t\t\t\t\t','')
            else:
                otherfeatures = None
            other_features_prague_sales.append(otherfeatures)

In [56]:
#extracting and cleaning neighbourhood
neighbourhood_prague_sales=[]
for apartments in listings_sales_prague:
    for listing in apartments:
        apt_neighbourhood=listing.find('span', class_="locality ng-binding").text.split(' ')[-1]
        neighbourhood_prague_sales.append(apt_neighbourhood)

In [57]:
#extracting and cleaning price
price_prague_sales=[]
for apartments in listings_sales_prague:
    for listing in apartments:
        apt_price=listing.find('span', class_="norm-price ng-binding").text.split('per')[0].replace('CZK','')
        price_prague_sales.append(apt_price)


In [58]:
prague_sales=pd.DataFrame({'types': types_prague_sales,'sizes':sizes_prague_sales,'units':units_prague_sales\
                           ,'streets':streets_prague_sales,'municipal_district':municipal_district_prague_sales,\
                           'neighbourhood':neighbourhood_prague_sales,'price':price_prague_sales,\
                           'other_features':other_features_prague_sales})
prague_sales

,types,sizes,units,streets,municipal_district,neighbourhood,price,other_features
0,2+1,85,m²,Janáčkovo nábřeží,Praha 5,Smíchov,13 900 000,Furnished
1,3+kt,84,m²,Pod Harfou,Praha 9,Vysočany,11 500 000,Parking
2,3+kt,92,m²,Fikerova,Praha 4,Modřany,11 850 000,New buildings
3,2+1,46,m²,Frýdlantská,Praha 8,Kobylisy,5 299 000,Loggia
4,4+kt,118,m²,Moravská,Praha 2,Vinohrady,23 944 513,Not furnished
...,...,...,...,...,...,...,...,...
2836,2+1,66,m²,Máchova,Praha 2,Vinohrady,8 900 000,None
2837,3+kt,70,m²,Kateřinské náměstí,Praha,Průhonic,8 599 500,None
2838,3+kt,70,m²,Praha 4 - Újezd u Průhonic,Praha 4,Průhonic,7 999 865,None
2839,4+kt,110,m²,Nad závodištěm,Praha 5,Chuchle,14 216 000,None


#### Performing EDA and Dropping outlier rows


In [59]:
print(prague_sales['types'].value_counts().to_string())

 2+kt    849
 3+kt    597
 1+kt    371
 3+1     338
 2+1     243
 4+kt    222
 1+1      79
 4+1      68
 5+kt     32
 5+1      16
 6 ro     15
 unus     11


In [60]:
print(prague_sales['sizes'].value_counts().to_string())

 49      61
 50      46
 52      44
 53      42
 43      41
 46      38
 70      37
 51      36
 44      35
 54      34
 58      34
 56      33
 45      33
 48      33
 77      33
 57      31
 42      31
 60      29
 65      29
 63      29
 78      28
 62      28
 33      28
 47      27
 55      26
 41      26
 68      26
 69      26
 82      25
 59      25
 35      25
 73      23
 40      23
 80      23
 32      23
 78      23
 67      22
 68      22
 29      22
 39      21
 55      21
 72      21
 79      20
 80      20
 27      20
 38      20
 34      20
 69      19
 65      19
 36      19
 85      19
 37      18
 86      18
 61      18
 67      18
 74      18
 71      18
 64      18
 92      18
 52      17
 73      17
 70      17
 53      17
 83      17
 76      16
 72      16
 28      16
 60      16
 26      16
 74      16
 31      16
 30      16
 75      15
 81      15
 71      15
 75      15
 94      15
 103     15
 54      14
 66      14
 84      14
 57      14
 76      14
 110

In [61]:
print(prague_sales['municipal_district'].value_counts().to_string())

Praha 5     476
Praha 10    451
Praha 4     430
Praha 9     389
Praha       205
Praha 6     203
Praha 8     202
Praha 3     168
Praha 2     111
Praha 7     110
Praha 1      96


In [62]:
print(prague_sales['streets'].value_counts().to_string())

K Jezeru                       93
Zimova                         39
Plzeňská                       22
Armády                         18
Suchdolské náměstí             16
Koněvova                       16
Strážní                        15
Pod Hrachovkou                 15
Olgy Havlové                   14
Kurandové                      14
Podbělohorská                  14
Radlická                       13
Mistřínská                     13
Sazovická                      13
Kolbenova                      13
V Háji                         12
Děčínská                       12
Na Neklance                    12
Tupolevova                     11
Lihovarská                     11
Týmlova                        11
Praha 10 - Praha 15            11
Počernická                     11
Záběhlická                     10
Oddechová                      10
Sokolovská                     10
Dlouhá                         10
Mánesova                       10
Pod Harfou                     10
Poděbradská   

In [63]:
prague_sales['neighbourhood'].value_counts().head(40)

Žižkov        158
Hostivař      126
Stodůlky      124
Smíchov       118
Město         114
Libeň         100
Vinohrady      99
Holešovice     87
Záběhlice      85
Hlubočepy      78
Vysočany       71
Kamýk          68
Letňany        67
Chodov         67
Vršovice       64
Strašnice      64
Nusle          56
Střížkov       54
Modřany        53
Michle         52
Břevnov        48
Zličín         47
Krč            46
Košíře         46
Most           43
Troja          39
Měcholupy      38
Uhříněves      36
Řepy           34
Bubeneč        33
Karlín         31
Kobylisy       29
Braník         29
Hloubětín      28
Suchdol        27
Praha          26
Dejvice        25
Prosek         25
Malešice       24
Kyje           23
Name: neighbourhood, dtype: int64

In [64]:
prague_sales['units'].value_counts()

m²    2841
Name: units, dtype: int64

In [65]:
prague_sales.drop(prague_sales[prague_sales['price'].str.contains('Information')].index, inplace=True)

In [66]:
prague_sales['types']=prague_sales['types'].str.replace(u'\xa0',u' ').str.strip().str.replace(' ','')

In [67]:
prague_sales.drop(prague_sales[prague_sales['streets']=='Praha'].index, inplace=True)

#### Checking and converting types

In [68]:
#checking types
print(prague_sales.dtypes)

types                 object
sizes                 object
units                 object
streets               object
municipal_district    object
neighbourhood         object
price                 object
other_features        object
dtype: object


In [69]:
#converting sizes to int
prague_sales['sizes']=prague_sales['sizes'].astype(int)

In [70]:
#converting prices to int
#prices contains xa0 (unicode character)
prague_sales['price']=prague_sales['price'].str.replace(u'\xa0',u' ').str.strip().str.replace(' ','')
prague_sales['price']=pd.to_numeric(prague_sales['price'])

In [71]:
prague_sales.head()

,types,sizes,units,streets,municipal_district,neighbourhood,price,other_features
0,2+1,85,m²,Janáčkovo nábřeží,Praha 5,Smíchov,13900000,Furnished
1,3+kt,84,m²,Pod Harfou,Praha 9,Vysočany,11500000,Parking
2,3+kt,92,m²,Fikerova,Praha 4,Modřany,11850000,New buildings
3,2+1,46,m²,Frýdlantská,Praha 8,Kobylisy,5299000,Loggia
4,4+kt,118,m²,Moravská,Praha 2,Vinohrady,23944513,Not furnished


In [72]:
prague_sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2714 entries, 0 to 2840
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   types               2714 non-null   object
 1   sizes               2714 non-null   int64 
 2   units               2714 non-null   object
 3   streets             2714 non-null   object
 4   municipal_district  2714 non-null   object
 5   neighbourhood       2714 non-null   object
 6   price               2714 non-null   int64 
 7   other_features      2233 non-null   object
dtypes: int64(2), object(6)
memory usage: 190.8+ KB


In [73]:
prague_sales.describe(include='all')

,types,sizes,units,streets,municipal_district,neighbourhood,price,other_features
count,2714,2714.000000,2714,2714,2714,2714,2.714000e+03,2233
unique,12,NaN,1,1244,11,104,NaN,66
top,2+kt,NaN,m²,K Jezeru,Praha 5,Žižkov,NaN,After reconstruction
freq,812,NaN,2714,93,447,144,NaN,248
mean,NaN,69.436625,NaN,NaN,NaN,NaN,8.499901e+06,NaN
std,NaN,33.612926,NaN,NaN,NaN,NaN,4.808253e+06,NaN
min,NaN,11.000000,NaN,NaN,NaN,NaN,2.280000e+05,NaN
25%,NaN,48.000000,NaN,NaN,NaN,NaN,5.490000e+06,NaN
50%,NaN,63.000000,NaN,NaN,NaN,NaN,6.995000e+06,NaN
75%,NaN,82.000000,NaN,NaN,NaN,NaN,9.993538e+06,NaN


#### Feature Engineering

In [74]:
#Calculating CostPerSquaremetre
prague_sales['CostPerSquaremetre']=prague_sales['price']/prague_sales['sizes']

In [75]:
prague_sales['CostPerSquaremetre']=prague_sales['CostPerSquaremetre'].round(0)

In [76]:
#does this apartment contain separate kitchen?
def kitchen(x):
    if 'kt' in  x:
        return '0'
    else:
        return '1'

In [77]:
#does this apartment contain separate kitchen?
prague_sales['Separate_kitchen']=prague_sales['types'].apply(kitchen)

In [78]:
#defining data category
prague_sales['data category']= 'prague_sales'

In [79]:
prague_sales.head()

,types,sizes,units,streets,municipal_district,neighbourhood,price,other_features,CostPerSquaremetre,Separate_kitchen,data category
0,2+1,85,m²,Janáčkovo nábřeží,Praha 5,Smíchov,13900000,Furnished,163529.0,1,prague_sales
1,3+kt,84,m²,Pod Harfou,Praha 9,Vysočany,11500000,Parking,136905.0,0,prague_sales
2,3+kt,92,m²,Fikerova,Praha 4,Modřany,11850000,New buildings,128804.0,0,prague_sales
3,2+1,46,m²,Frýdlantská,Praha 8,Kobylisy,5299000,Loggia,115196.0,1,prague_sales
4,4+kt,118,m²,Moravská,Praha 2,Vinohrady,23944513,Not furnished,202920.0,0,prague_sales


### Extracting rent data for Brno

In [82]:
#scraping rent data for Brno
#importing url using selenium
options = Options()
options.add_argument('--disable-blink-features=AutomationControlled')
driver.set_script_timeout("600")
driver = webdriver.Chrome()
url_list_rent_brno = 'https://www.sreality.cz/en/search/to-rent/apartments/brno,brno-venkov?page='
urls_rent_brno=[]
page_checking = True
page_count = 1
#looping through pages until the last page
while page_checking:
    url_rent_brno = url_list_rent_brno + str(page_count)
    driver.get(url_rent_brno)
    driver.maximize_window()
    time.sleep(5)
    response = driver.page_source.encode('utf-8').strip()
    soup = BeautifulSoup(response, 'lxml')
    
    last_page = soup.find('p', class_="status-text ng-binding")
    
    if last_page is not None:
        page_checking = False
    
    else:
        urls_rent_brno.append(url_rent_brno)
        page_count = page_count+1
driver.quit()
print(urls_rent_brno)

['https://www.sreality.cz/en/search/to-rent/apartments/brno,brno-venkov?page=1', 'https://www.sreality.cz/en/search/to-rent/apartments/brno,brno-venkov?page=2', 'https://www.sreality.cz/en/search/to-rent/apartments/brno,brno-venkov?page=3', 'https://www.sreality.cz/en/search/to-rent/apartments/brno,brno-venkov?page=4', 'https://www.sreality.cz/en/search/to-rent/apartments/brno,brno-venkov?page=5', 'https://www.sreality.cz/en/search/to-rent/apartments/brno,brno-venkov?page=6', 'https://www.sreality.cz/en/search/to-rent/apartments/brno,brno-venkov?page=7', 'https://www.sreality.cz/en/search/to-rent/apartments/brno,brno-venkov?page=8', 'https://www.sreality.cz/en/search/to-rent/apartments/brno,brno-venkov?page=9', 'https://www.sreality.cz/en/search/to-rent/apartments/brno,brno-venkov?page=10', 'https://www.sreality.cz/en/search/to-rent/apartments/brno,brno-venkov?page=11', 'https://www.sreality.cz/en/search/to-rent/apartments/brno,brno-venkov?page=12', 'https://www.sreality.cz/en/search/t

In [83]:
#looping through each page to extract listings
listings_rent_brno=[]
for webpages in urls_rent_brno:
    #importing url using selenium
    driver = webdriver.Chrome()
    driver.get(webpages)
    time.sleep(5)
    responses = driver.page_source.encode('utf-8').strip()
    soup = BeautifulSoup(responses, 'lxml')
    #listingss = soup.find_all('div')
    li = soup.find_all('div', class_='info clear ng-scope')
    listings_rent_brno.append(li)
driver.quit()

### Extracting, cleaning, and transforming required fields

#### Looping through to get individual properties/fields


In [84]:
#extracting and cleaning  apartment types 
types_brno_rent=[]
for apartments in listings_rent_brno:
    for listing in apartments:
        apt_type=listing.find('span',class_="name ng-binding").text.split('apartment')[-1][:5]
        types_brno_rent.append(apt_type)

In [85]:
#extracting and cleaning apartment sizes 
sizes_brno_rent=[]
for apartments in listings_rent_brno:
    for listing in apartments:   
        apt_size=listing.find('span',class_="name ng-binding").text.replace('+1',',').replace('kt',',').split(',')\
        [-1].split('m²')[0].split('unusual')[-1].split('plus')[-1]
        sizes_brno_rent.append(apt_size)

In [86]:
#extracting and cleaning units 
units_brno_rent=[]
for apartments in listings_rent_brno:
    for listing in apartments:   
        apt_units=listing.find('span',class_="name ng-binding").text.replace('+1',',').replace('kt',',')\
        .rpartition('m²')[1]
        [-1][3:]
        units_brno_rent.append(apt_units)

In [87]:
#extracting and cleaning streets
streets_brno_rent=[]
for apartments in listings_rent_brno:
    for listing in apartments:    
        apt_street=listing.find('span', class_="locality ng-binding").text.split(',',1)[0]
        streets_brno_rent.append(apt_street)

In [88]:
#extracting and cleaning municipal_district
municipal_district_brno_rent=[]
for apartments in listings_rent_brno:
    for listing in apartments:
        apt_municipal_district=listing.find('span', class_="locality ng-binding").text.split(',',1)[-1].split('-')[0]
        municipal_district_brno_rent.append(apt_municipal_district)

In [89]:
#extracting and cleaning features
other_features_brno_rent=[]
for apartments in listings_rent_brno:
    for listing in apartments:
        features=[]
        feature=listing.find('span', class_="label ng-binding ng-scope")
        features.append(feature)
        for feat in features:
            if feat is not None:
                otherfeatures = feat.text.replace('\n\t\t\t\t\t','')
            else:
                otherfeatures = None
            other_features_brno_rent.append(otherfeatures)

In [90]:
#extracting and cleaning neighbourhood
neighbourhood_brno_rent=[]
for apartments in listings_rent_brno:
    for listing in apartments:
        apt_neighbourhood=listing.find('span', class_="locality ng-binding").text.split(' ')[-1]
        neighbourhood_brno_rent.append(apt_neighbourhood)

In [91]:
#extracting and cleaning price
price_brno_rent=[]
for apartments in listings_rent_brno:
    for listing in apartments:
        apt_price=listing.find('span', class_="norm-price ng-binding").text.split('per')[0].replace('CZK','')
        price_brno_rent.append(apt_price)

In [92]:
brno_rent=pd.DataFrame({'types': types_brno_rent,'sizes':sizes_brno_rent,'units':units_brno_rent,'streets'\
                         :streets_brno_rent,'municipal_district':municipal_district_brno_rent,'neighbourhood'\
                         :neighbourhood_brno_rent,'price':price_brno_rent,'other_features':other_features_brno_rent})
brno_rent.head(20)

,types,sizes,units,streets,municipal_district,neighbourhood,price,other_features
0,3+1,80,m²,Teyschlova,Brno,Bystrc,19 000,Loggia
1,1+kt,23,m²,Údolní,Brno,Veveří,11 500,Furnished
2,2+kt,45,m²,Kociánka,Brno,Pole,14,Collective owenship
3,4+kt,83,m²,Ponětovická,Brno,Slatina,30 000,Terrace
4,2+kt,51,m²,Trnitá,Brno,Trnitá,17 000,New buildings
5,3+1,105,m²,Grohova,Brno,Veveří,26 000,Not furnished
6,2+kt,63,m²,K Rybníku,Brno,Medlánky,16 000,New buildings
7,2+1,67,m²,Jílová,Brno,Štýřice,13 000,Concrete
8,2+1,56,m²,Židlochovice,district Brno,Brno-venkov,14 500,Parking
9,To re,roomshare 25,m²,Zdráhalova,Brno,Pole,9 500,Post office 4 min walk


#### Performing EDA and Dropping outlier rows

In [93]:
brno_rent['types'].value_counts()

 1+kt    140
 2+kt    120
 2+1      78
 1+1      57
 3+kt     43
 3+1      36
 4+1      12
 4+kt      8
To re      7
 5+kt      2
 unus      1
 5+1       1
Name: types, dtype: int64

In [94]:
print(brno_rent['sizes'].value_counts().to_string())

 30               18
 45               13
 32               11
 24               10
 40                9
 52                9
 50                9
 26                9
 60                9
 34                9
 40                8
 53                8
 50                8
 33                8
 37                7
 28                7
 44                7
 57                7
 80                6
 60                6
 55                6
 70                6
 55                6
 42                6
 41                5
 35                5
 25                5
 75                5
 47                5
 56                5
 20                5
 58                4
 19                4
 66                4
 54                4
 42                4
 96                4
 72                4
 59                4
 85                4
 31                4
 73                4
 62                4
 46                4
 52                4
 36                4
 43                4
 22          

In [95]:
print(brno_rent['streets'].value_counts().to_string())

Bratislavská               11
Václavská                  11
Palackého třída             8
Holzova                     8
Hybešova                    8
Minská                      6
Spolková                    6
Střední                     6
Královopolská               5
Hvězdová                    5
Tkalcovská                  5
Placzkova                   5
Štefánikova                 5
Purkyňova                   5
Kotlářská                   4
Halasovo náměstí            4
Pellicova                   4
Pekařská                    4
Lidická                     4
Božetěchova                 4
Rybníček                    4
Trnitá                      4
Milady Horákové             3
Kleštínek                   3
K Babě                      3
Znojemská                   3
Chvalovka                   3
třída Generála Píky         3
Srbská                      3
náměstí Svobody             3
Teyschlova                  3
Tyršova                     3
Gellnerova                  3
Botanická 

In [96]:
brno_rent['municipal_district'].value_counts()

 Brno                     425
 district Brno             25
 Brno                      10
 Kuřim                      8
 Pohořelice                 5
 Rosice                     5
 Tišnov                     4
 Ivančice                   3
 Rajhrad                    2
 Vranovice                  2
 Hrušovany u Brna           2
 Modřice                    2
 Zastávka                   1
 Židlochovice               1
 Moravské Knínice           1
 Bílovice nad Svitavou      1
 Domašov                    1
 Moravany                   1
 Pozořice                   1
 Ostopovice                 1
 Říčany                     1
 Lelekovice                 1
 Dolní Kounice              1
 Šlapanice                  1
Name: municipal_district, dtype: int64

In [97]:
brno_rent['neighbourhood'].value_counts().head(40)

Pole           51
Brno           51
Veveří         40
Zábrdovice     39
Brno-město     30
Žabovřesky     25
Židenice       23
Lesná          22
Líšeň          17
Bystrc         15
Trnitá         15
Ponava         14
Řečkovice      11
Brno-venkov    10
Štýřice        10
Medlánky        9
Pisárky         8
Kuřim           8
Žebětín         8
Lískovec        7
Jundrov         7
Slatina         6
Rosice          5
Pohořelice      5
Jehnice         5
Černovice       4
Tišnov          4
Kohoutovice     4
Bohunice        4
Maloměřice      3
Sadová          3
Husovice        3
Ivančice        3
Komín           3
Vranovice       2
Bosonohy        2
Rajhrad         2
Ivanovice       2
Brno-sever      2
Brna            2
Name: neighbourhood, dtype: int64

In [98]:
brno_rent.drop(brno_rent[brno_rent['price'].str.contains('Information')].index, inplace=True)

In [99]:
brno_rent.drop(brno_rent[brno_rent['sizes'].str.contains('room')].index, inplace=True)

In [100]:
brno_rent['types']=brno_rent['types'].str.replace(u'\xa0',u' ').str.strip().str.replace(' ','')

#### Checking and converting types

In [101]:
#checking types
print(brno_rent.dtypes)

types                 object
sizes                 object
units                 object
streets               object
municipal_district    object
neighbourhood         object
price                 object
other_features        object
dtype: object


In [102]:
#converting sizes to int
brno_rent['sizes']=brno_rent['sizes'].str.replace(u'\xa0',u' ').str.strip().str.replace(' ','')
brno_rent['sizes']=pd.to_numeric(brno_rent['sizes'])

In [103]:
#converting prices to int
#prices contains xa0 (unicode character)
brno_rent['price']=brno_rent['price'].str.replace(u'\xa0',u' ').str.strip().str.replace(' ','')
brno_rent['price']=pd.to_numeric(brno_rent['price'])

In [104]:
brno_rent.head()

,types,sizes,units,streets,municipal_district,neighbourhood,price,other_features
0,3+1,80,m²,Teyschlova,Brno,Bystrc,19000,Loggia
1,1+kt,23,m²,Údolní,Brno,Veveří,11500,Furnished
2,2+kt,45,m²,Kociánka,Brno,Pole,14,Collective owenship
3,4+kt,83,m²,Ponětovická,Brno,Slatina,30000,Terrace
4,2+kt,51,m²,Trnitá,Brno,Trnitá,17000,New buildings


#### Feature Engineering

In [105]:
#Calculating CostPerSquaremetre
brno_rent['CostPerSquaremetre']= brno_rent['price']/brno_rent['sizes']

In [106]:
brno_rent['CostPerSquaremetre']=brno_rent['CostPerSquaremetre'].round(0)

In [107]:
#does this apartment contain separate kitchen?
brno_rent['Separate_kitchen']=brno_rent['types'].apply(kitchen)

In [108]:
#defining data category
brno_rent['data category']= 'brno_rent'

In [109]:
brno_rent.head()

,types,sizes,units,streets,municipal_district,neighbourhood,price,other_features,CostPerSquaremetre,Separate_kitchen,data category
0,3+1,80,m²,Teyschlova,Brno,Bystrc,19000,Loggia,238.0,1,brno_rent
1,1+kt,23,m²,Údolní,Brno,Veveří,11500,Furnished,500.0,0,brno_rent
2,2+kt,45,m²,Kociánka,Brno,Pole,14,Collective owenship,0.0,0,brno_rent
3,4+kt,83,m²,Ponětovická,Brno,Slatina,30000,Terrace,361.0,0,brno_rent
4,2+kt,51,m²,Trnitá,Brno,Trnitá,17000,New buildings,333.0,0,brno_rent


In [110]:
brno_rent.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 495 entries, 0 to 504
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   types               495 non-null    object 
 1   sizes               495 non-null    int64  
 2   units               495 non-null    object 
 3   streets             495 non-null    object 
 4   municipal_district  495 non-null    object 
 5   neighbourhood       495 non-null    object 
 6   price               495 non-null    int64  
 7   other_features      385 non-null    object 
 8   CostPerSquaremetre  495 non-null    float64
 9   Separate_kitchen    495 non-null    object 
 10  data category       495 non-null    object 
dtypes: float64(1), int64(2), object(8)
memory usage: 46.4+ KB


### Extracting rent data for Prague


In [113]:
#scraping rent data for Prague
#importing url using selenium
options = Options()
options.add_argument('--disable-blink-features=AutomationControlled')
driver.set_script_timeout("600")
driver = webdriver.Chrome()
url_list_rent_prague = 'https://www.sreality.cz/en/search/to-rent/apartments/praha?page='
urls_rent_prague=[]
page_checking = True
page_count = 1

while page_checking:
    url_rent_prague = url_list_rent_prague + str(page_count)
    driver.get(url_rent_prague)
    driver.maximize_window()
    time.sleep(5)
    response = driver.page_source.encode('utf-8').strip()
    soup = BeautifulSoup(response, 'lxml')
    
    last_page = soup.find('p', class_="status-text ng-binding")
    
    if last_page is not None:
        page_checking = False
    
    else:
        urls_rent_prague.append(url_rent_prague)
        page_count = page_count+1
driver.quit()
print(urls_rent_prague)

['https://www.sreality.cz/en/search/to-rent/apartments/praha?page=1', 'https://www.sreality.cz/en/search/to-rent/apartments/praha?page=2', 'https://www.sreality.cz/en/search/to-rent/apartments/praha?page=3', 'https://www.sreality.cz/en/search/to-rent/apartments/praha?page=4', 'https://www.sreality.cz/en/search/to-rent/apartments/praha?page=5', 'https://www.sreality.cz/en/search/to-rent/apartments/praha?page=6', 'https://www.sreality.cz/en/search/to-rent/apartments/praha?page=7', 'https://www.sreality.cz/en/search/to-rent/apartments/praha?page=8', 'https://www.sreality.cz/en/search/to-rent/apartments/praha?page=9', 'https://www.sreality.cz/en/search/to-rent/apartments/praha?page=10', 'https://www.sreality.cz/en/search/to-rent/apartments/praha?page=11', 'https://www.sreality.cz/en/search/to-rent/apartments/praha?page=12', 'https://www.sreality.cz/en/search/to-rent/apartments/praha?page=13', 'https://www.sreality.cz/en/search/to-rent/apartments/praha?page=14', 'https://www.sreality.cz/en/

In [114]:
listings_rent_prague=[]
for webpages in urls_rent_prague:
    #importing url using selenium
    driver = webdriver.Chrome()
    driver.get(webpages)
    time.sleep(5)
    responses = driver.page_source.encode('utf-8').strip()
    soup = BeautifulSoup(responses, 'lxml')
    #listingss = soup.find_all('div')
    li = soup.find_all('div', class_='info clear ng-scope')
    listings_rent_prague.append(li)
driver.quit()

### Extracting, cleaning, and transforming required fields


#### Looping through to get individual properties/fields


In [115]:
#extracting and cleaning  apartment types 
types_prague_rent=[]
for apartments in listings_rent_prague:
    for listing in apartments:
        apt_type=listing.find('span',class_="name ng-binding").text.split('apartment')[-1][:5]
        types_prague_rent.append(apt_type)

In [116]:
#extracting and cleaning apartment sizes 
sizes_prague_rent=[]
for apartments in listings_rent_prague:
    for listing in apartments:   
        apt_size=listing.find('span',class_="name ng-binding").text.replace('+1',',').replace('kt',',').split(',')\
        [-1].split('m²')[0].split('unusual')[-1].split('plus')[-1].replace(u'\xa0',u' ')
        sizes_prague_rent.append(apt_size) 

In [117]:
#extracting and cleaning units 
units_prague_rent=[]
for apartments in listings_rent_prague:
    for listing in apartments:   
        apt_units=listing.find('span',class_="name ng-binding").text.replace('+1',',').replace('kt',',')\
        .rpartition('m²')[1]
        [-1][3:]
        units_prague_rent.append(apt_units)

In [118]:
#extracting and cleaning streets
streets_prague_rent=[]
for apartments in listings_rent_prague:
    for listing in apartments:    
        apt_street=listing.find('span', class_="locality ng-binding").text.split(',',1)[0]
        streets_prague_rent.append(apt_street)

In [119]:
#extracting and cleaning municipal_district
municipal_district_prague_rent=[]
for apartments in listings_rent_prague:
    for listing in apartments:
        apt_municipal_district=listing.find('span', class_="locality ng-binding").text.split(',',1)[-1].split('-')[0]
        municipal_district_prague_rent.append(apt_municipal_district.strip())

In [120]:
#extracting and cleaning features
other_features_prague_rent=[]
for apartments in listings_rent_prague:
    for listing in apartments:
        features=[]
        feature=listing.find('span', class_="label ng-binding ng-scope")
        features.append(feature)
        for feat in features:
            if feat is not None:
                otherfeatures = feat.text.replace('\n\t\t\t\t\t','')
            else:
                otherfeatures = None
            other_features_prague_rent.append(otherfeatures)

In [121]:
#extracting and cleaning neighbourhood
neighbourhood_prague_rent=[]
for apartments in listings_rent_prague:
    for listing in apartments:
        apt_neighbourhood=listing.find('span', class_="locality ng-binding").text.split(' ')[-1]
        neighbourhood_prague_rent.append(apt_neighbourhood)

In [122]:
#extracting and cleaning price
price_prague_rent=[]
for apartments in listings_rent_prague:
    for listing in apartments:
        apt_price=listing.find('span', class_="norm-price ng-binding").text.split('per')[0].replace('CZK','')
        price_prague_rent.append(apt_price)

In [123]:
prague_rent=pd.DataFrame({'types': types_prague_rent,'sizes':sizes_prague_rent,'units':units_prague_rent\
                           ,'streets':streets_prague_rent,'municipal_district':municipal_district_prague_rent,\
                          'neighbourhood':neighbourhood_prague_rent,'price':price_prague_rent,\
                          'other_features':other_features_prague_rent})
prague_rent

,types,sizes,units,streets,municipal_district,neighbourhood,price,other_features
0,2+kt,50,m²,Pod Vyšehradem,Praha 4,Podolí,21 000,Garage
1,2+1,47,m²,Odborů,Praha 2,Město,24 000,Grocery 5 min walk
2,3+kt,70,m²,Tomáškova,Praha 5,Smíchov,25 000,Cellar
3,2+1,40,m²,Souběžná III,Praha 5,Jinonice,15 000,Terrace
4,3+kt,115,m²,Radimova,Praha 6,Břevnov,36 000,New buildings
...,...,...,...,...,...,...,...,...
1950,3+kt,109,m²,Filmařská,Praha 5,Hlubočepy,33 000,None
1951,3+kt,88,m²,Toužimská,Praha 9,Kbely,19 500,Balcony
1952,3+kt,107,m²,Italská,Praha 2,Vinohrady,61 321,Balcony
1953,5+1,210,m²,Pod Kaštany,Praha 6,Bubeneč,80 000,Cellar


#### Performing EDA and Dropping outlier rows

In [124]:
prague_rent['types'].value_counts()

 2+kt    636
 3+kt    325
 1+kt    321
 2+1     169
 3+1     162
 1+1     136
 4+kt     87
 4+1      47
To re     25
 5+kt     16
 5+1      15
 unus     15
 6 ro      1
Name: types, dtype: int64

In [125]:
print(prague_rent['sizes'].value_counts().to_string())

 50                67
 45                55
 40                53
 55                52
 60                45
 52                39
 48                34
 80                34
 70                33
 30                32
 53                32
 54                31
 58                31
 43                31
 47                30
 35                30
 38                27
 57                27
 32                27
 42                27
 33                26
 25                25
 36                25
 56                25
 27                24
 44                24
 34                24
 46                24
 65                24
 75                23
 100               22
 85                22
 51                22
 78                20
 37                20
 62                19
 94                18
 68                18
 63                18
 73                18
 39                17
 108               17
 72                17
 49                17
 71                17
 90       

In [126]:
print(prague_rent['streets'].value_counts().to_string())

Italská                        31
Máchova                        22
Holečkova                      22
Laubova                        21
Sázavská                       17
Freyova                        14
U Zvonařky                     13
Vlašská                        13
Mánesova                       13
Pařížská                       12
Sokolovská                     12
Washingtonova                  12
Kurta Konráda                   9
Truhlářská                      9
Řehořova                        9
Opletalova                      9
Hořejší nábřeží                 9
Londýnská                       9
Slezská                         9
Ruská                           8
Újezd                           8
Petrská                         8
Čelakovského sady               8
Anny Letenské                   8
Pod Harfou                      8
Štěpánská                       7
Michelská                       7
Křižíkova                       7
Dělnická                        7
Smržových     

In [127]:
prague_rent['municipal_district'].value_counts()

Praha 5     290
Praha 4     252
Praha 2     232
Praha 10    187
Praha 1     186
Praha 3     184
Praha 6     151
Praha 9     151
Praha 8     128
Praha       113
Praha 7      81
Name: municipal_district, dtype: int64

In [128]:
prague_rent['neighbourhood'].value_counts().head(40)

Vinohrady     255
Město         203
Žižkov        118
Smíchov       114
Nusle          66
Libeň          66
Vršovice       65
Holešovice     61
Strana         51
Vysočany       51
Karlín         48
Stodůlky       46
Hlubočepy      42
Michle         41
Strašnice      36
Břevnov        35
Dejvice        35
Bubeneč        34
Košíře         33
Modřany        32
Krč            30
Záběhlice      28
Měcholupy      22
Podolí         19
Troja          18
Uhříněves      16
Kamýk          15
Letňany        15
Chodov         15
Ruzyně         14
Jinonice       14
Braník         13
Hostivař       13
Veleslavín     12
Hloubětín      12
Josefov        11
Řepy           10
Most           10
5               9
Kobylisy        9
Name: neighbourhood, dtype: int64

In [129]:
prague_rent['other_features'].value_counts()

New buildings              244
After reconstruction       244
Balcony                    119
Terrace                    106
Partly funished             96
Not furnished               94
Loggia                      92
Concrete                    77
Parking                     60
Collective owenship         58
Garage                      54
Cellar                      51
Medic 6 min walk            20
Furnished                   20
Grocery 6 min walk          17
Grocery 5 min walk          16
Grocery 7 min walk          10
Train 5 min walk            10
Subway 4 min walk            9
State owenship               9
Medic 7 min walk             9
Medic 3 min walk             8
Medic 4 min walk             7
Medic 5 min walk             6
Subway 5 min walk            6
Medic 1 min walk             6
Medic 2 min walk             5
Subway 6 min walk            5
Train 6 min walk             5
Subway 3 min walk            5
Train 7 min walk             4
Subway 2 min walk            3
Grocery 

In [130]:
prague_rent.drop(prague_rent[prague_rent['sizes'].str.contains('r')].index,inplace=True)

In [131]:
prague_rent.drop(prague_rent[prague_rent['price'].str.contains('Information')].index, inplace=True)

In [132]:
prague_rent['types']=prague_rent['types'].str.replace(u'\xa0',u' ').str.strip().str.replace(' ','')

#### Checking and converting types

In [133]:
#checking types
print(prague_rent.dtypes)

types                 object
sizes                 object
units                 object
streets               object
municipal_district    object
neighbourhood         object
price                 object
other_features        object
dtype: object


In [134]:
#converting sizes to int
prague_rent['sizes']=prague_rent['sizes'].str.replace(u'\xa0',u' ').str.strip().str.replace(' ','')
prague_rent['sizes']=pd.to_numeric(prague_rent['sizes'])

In [135]:
#converting prices to int
#prices contains xa0 (unicode character)
prague_rent['price']=prague_rent['price'].str.replace(u'\xa0',u' ').str.strip().str.replace(' ','')
prague_rent['price']=pd.to_numeric(prague_rent['price'])

In [136]:
prague_rent.head()

,types,sizes,units,streets,municipal_district,neighbourhood,price,other_features
0,2+kt,50,m²,Pod Vyšehradem,Praha 4,Podolí,21000,Garage
1,2+1,47,m²,Odborů,Praha 2,Město,24000,Grocery 5 min walk
2,3+kt,70,m²,Tomáškova,Praha 5,Smíchov,25000,Cellar
3,2+1,40,m²,Souběžná III,Praha 5,Jinonice,15000,Terrace
4,3+kt,115,m²,Radimova,Praha 6,Břevnov,36000,New buildings


#### Feature Engineering


In [137]:
#Calculating CostPerSquaremetre
prague_rent['CostPerSquaremetre']= prague_rent['price']/prague_rent['sizes']

In [138]:
prague_rent['CostPerSquaremetre']=prague_rent['CostPerSquaremetre'].round(0)

In [139]:
#does this apartment contain separate kitchen?
prague_rent['Separate_kitchen']=prague_rent['types'].apply(kitchen)

In [140]:
#defining data category
prague_rent['data category']= 'prague_rent'

In [141]:
prague_rent.head()

,types,sizes,units,streets,municipal_district,neighbourhood,price,other_features,CostPerSquaremetre,Separate_kitchen,data category
0,2+kt,50,m²,Pod Vyšehradem,Praha 4,Podolí,21000,Garage,420.0,0,prague_rent
1,2+1,47,m²,Odborů,Praha 2,Město,24000,Grocery 5 min walk,511.0,1,prague_rent
2,3+kt,70,m²,Tomáškova,Praha 5,Smíchov,25000,Cellar,357.0,0,prague_rent
3,2+1,40,m²,Souběžná III,Praha 5,Jinonice,15000,Terrace,375.0,1,prague_rent
4,3+kt,115,m²,Radimova,Praha 6,Břevnov,36000,New buildings,313.0,0,prague_rent


#### Loading data to CSV and importing to Tableau for More visualisation

In [157]:
#unioning files by city
brno_all_data=pd.concat([brno_sales,brno_rent], axis=0).reset_index()

In [158]:
prague_all_data=pd.concat([prague_sales,prague_rent], axis=0).reset_index()

In [159]:
#Loading file 1
brno_all_data.to_excel('brno_all_data.xlsx', index=False)

In [160]:
#Loading file 2
prague_all_data.to_excel('prague_all_data.xlsx', index=False)